# TP 1 - NLP - Word2Vect


<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


## Alumno:

### Domenje Carlos R.


### Librerias a utilizar

In [3]:
import numpy as np
import pandas as pd

In [4]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [5]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [6]:
# Esta función nos retorna una lista de todos los términos que se encuentran en un corpus.
def getListOfTerms(corpus):
    terms_l = np.empty([corpus.shape[0],],dtype=object)
    for i in range(len(corpus)):
        terms_l[i] = corpus[i].split(" ")

    return terms_l

In [7]:
def getVocabulary(list_of_terms):
    # Creo un set de objetos que van a ser unicos.
    voc = set()
    # Recorro la lista de entrada y me quedo solo con los que no se repiten.
    for item in list_of_terms:
        voc =  voc  | set(item)
    voc = np.array(list(voc))
    # Debemos retornar un array de numpy, asi que convierto la lista.
    return voc

In [8]:
list_terms = getListOfTerms(corpus)
vocabulary = getVocabulary(list_terms)

print("Términos: ", list_terms)

print("Vocabulario: ",vocabulary)


Términos:  [list(['que', 'dia', 'es', 'hoy'])
 list(['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes'])
 list(['martes', 'muchas', 'gracias'])]
Vocabulario:  ['hoy' 'que' 'gracias' 'es' 'martes' 'de' 'dia' 'el' 'muchas']


### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [12]:
def oneHotEncoding(list_term):
    voc = getVocabulary(list_term)
    df = pd.DataFrame(np.zeros((len(list_term),len(voc))) ,columns=voc,dtype=np.int8)
    for i,lt in enumerate(list_term):
        for item in voc:
            df.loc[i, item]=np.isin(item, lt).astype(int)  
    return df

one_hot_encoding = oneHotEncoding(list_terms)
print("OneHotEncoding:\n", one_hot_encoding)

OneHotEncoding:
    hoy  que  gracias  es  martes  de  dia  el  muchas
0    1    1        0   1       0   0    1   0       0
1    1    0        0   1       1   1    1   1       0
2    0    0        1   0       1   0    0   0       1


### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [14]:
def frecuencyVectors(list_term):
    voc = getVocabulary(list_term)
    df = pd.DataFrame(np.zeros((len(list_term),len(voc))) ,columns=voc,dtype=np.int8)
    for i,lt in enumerate(list_term):
        for item in voc:
            df.loc[i, item] = lt.count(item)
    return df

freq_vector = frecuencyVectors(list_terms)
print("Vectores de Frecuencia:\n", freq_vector)

Vectores de Frecuencia:
    hoy  que  gracias  es  martes  de  dia  el  muchas
0    1    1        0   1       0   0    1   0       0
1    1    0        0   1       2   1    1   1       0
2    0    0        1   0       1   0    0   0       1


### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [17]:
def getIdf(list_term):
    oneHot = oneHotEncoding(list_term)
    idf = np.log10(len(list_term) / oneHot.sum(axis=0))
    return idf

def getTFIDF(list_term):
    tf_idf = getIdf(list_term) * frecuencyVectors(list_term)
    return tf_idf

### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [20]:
def cosSimilList(corpus,indice):
    simil_list = pd.DataFrame()
    simil_list.index = corpus
    term_list = getListOfTerms(corpus)
    TFIDF = getTFIDF(term_list)
    TFIDF_i = TFIDF.iloc[[indice]]
    for idx,lt in enumerate(term_list):
         simil_list.loc[corpus[idx],"distancia"]=cosine_similarity(TFIDF_i,TFIDF.iloc[idx])
    
    simil_list = simil_list.drop(corpus[indice]).sort_values(by="distancia",ascending=False)
    return simil_list
        
def getDistance(corpus,indice):
    print("Distancia :\n".format(corpus[indice]))
    print(cosSimilList(corpus,indice))
    print("\n")
    
getDistance(corpus,0)
getDistance(corpus,1)
getDistance(corpus,2) 


Distancia :

                                distancia
martes el dia de hoy es martes   0.200342
martes muchas gracias            0.000000


Distancia :

                       distancia
que dia es hoy          0.200342
martes muchas gracias   0.108457


Distancia :

                                distancia
martes el dia de hoy es martes   0.108457
que dia es hoy                   0.000000


